<a href="https://colab.research.google.com/github/PradeepSarangadharan/people-ai-consumewise/blob/main/SmartLabelReader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install sqlalchemy pandas requests scikit-learn opencv-python-headless pytesseract beautifulsoup4 ipywidgets
!apt-get install -y tesseract-ocr
!pip install pytesseract
!pip install openai==0.27.0 opencv-python-headless pytesseract


  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,073 kB/s)
Selecting previously unselected package tesseract-

In [2]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from zipfile import ZipFile
import cv2
import pytesseract
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from bs4 import BeautifulSoup
from sqlalchemy import create_engine, Column, Integer, String, Float, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from google.colab import files
import logging
from IPython.display import display, HTML
import json
from google.colab import output
import openai
from google.colab import files
from io import BytesIO
import re

# openai.api_key = "secret_key" (Input the API key to access Chat Module )

In [3]:
# Set up logging
logging.getLogger('sqlalchemy.engine').setLevel(logging.WARNING)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Force output to display
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Base = declarative_base()

class Product(Base):
    __tablename__ = 'products'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    food_group_id = Column(String)
    food_code = Column(String)
    description = Column(Text)
    calories = Column(String)
    protein = Column(String)
    carbohydrates = Column(String)
    fat = Column(String)
    sugar = Column(String)
    sodium = Column(String)

<ipython-input-3-9f398b1ccd3f>:10: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [4]:
class HealthAnalysis:
    def __init__(self):
        self.nutrition_model = self.train_nutrition_model()
        self.processing_model = self.train_processing_model()
        self.diet_compliance = {
            'vegan': ['milk', 'eggs', 'meat', 'fish'],
            'vegetarian': ['meat', 'fish'],
            'keto': ['sugar', 'wheat', 'rice'],
            'gluten_free': ['wheat', 'barley', 'rye']
        }
        self.allergens = ['peanuts', 'tree nuts', 'milk', 'eggs', 'fish', 'shellfish', 'soy', 'wheat']

    def train_nutrition_model(self):
        # Enhanced sample data for nutrition model
        np.random.seed(42)
        n_samples = 1000
        data = pd.DataFrame({
            'calories': np.random.randint(50, 800, n_samples),
            'fat': np.random.randint(0, 50, n_samples),
            'sugar': np.random.randint(0, 30, n_samples),
            'sodium': np.random.randint(0, 1000, n_samples),
        })
        # Simplified health rating calculation
        data['health_rating'] = 5 - (
            (data['calories'] // 200) +
            (data['fat'] // 10) +
            (data['sugar'] // 10) +
            (data['sodium'] // 200)
        ).clip(0, 4)

        X = data[['calories', 'fat', 'sugar', 'sodium']]
        y = data['health_rating']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        print("Nutrition Model Performance:")
        print(classification_report(y_test, model.predict(X_test)))

        return model

    def train_processing_model(self):
        # Enhanced sample data for processing model
        ingredients_list = [
            'water', 'sugar', 'salt', 'flour', 'vegetable oil', 'milk', 'eggs',
            'artificial flavors', 'preservatives', 'natural flavors', 'vitamins',
            'minerals', 'food coloring', 'emulsifiers', 'stabilizers'
        ]
        n_samples = 1000
        data = pd.DataFrame({
            'ingredients': [
                ', '.join(np.random.choice(ingredients_list, np.random.randint(3, 10), replace=False))
                for _ in range(n_samples)
            ]
        })

        # Simplified processing level calculation
        data['processing_level'] = data['ingredients'].apply(lambda x: len(x.split(', '))).clip(1, 4)

        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(data['ingredients'])
        y = data['processing_level']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        print("Processing Model Performance:")
        print(classification_report(y_test, model.predict(X_test)))

        return model, vectorizer

    def analyze_nutrition(self, nutrition_info):
        # Simplified nutrition analysis
        analysis = {}
        for nutrient, value in nutrition_info.items():
            numeric_value = self.extract_numeric_value(value)
            if numeric_value is not None:
                if nutrient == 'calories':
                    analysis[nutrient] = "High" if numeric_value > 300 else "Low"
                elif nutrient in ['total_fat', 'saturated_fat']:
                    analysis[nutrient] = "High" if numeric_value > 20 else "Low"
                elif nutrient == 'sodium':
                    analysis[nutrient] = "High" if numeric_value > 500 else "Low"
                elif nutrient == 'total_sugars':
                    analysis[nutrient] = "High" if numeric_value > 20 else "Low"
        return analysis

    def analyze_processing(self, ingredients):
        ingredients_lower = ingredients.lower()
        if any(additive in ingredients_lower for additive in ['artificial', 'preservative', 'flavor enhancer']):
            return "Highly Processed"
        elif len(ingredients.split(',')) > 5:
            return "Moderately Processed"
        else:
            return "Minimally Processed"

    def check_harmful_ingredients(self, ingredients):
        harmful = ['high fructose corn syrup', 'artificial colors', 'artificial flavors', 'MSG']
        return [ing for ing in harmful if ing in ingredients.lower()]

    def check_diet_compliance(self, ingredients, diet):
        ingredients_lower = ingredients.lower()
        non_compliant = [ing for ing in self.diet_compliance[diet] if ing in ingredients_lower]
        return len(non_compliant) == 0

    def check_allergens(self, ingredients):
        ingredients_lower = ingredients.lower()
        return [allergen for allergen in self.allergens if allergen in ingredients_lower]

    def extract_numeric_value(self, value_str):
        if isinstance(value_str, (int, float)):
            return value_str
        match = re.search(r'(\d+(\.\d+)?)', str(value_str))
        return float(match.group(1)) if match else None

    def analyze_product(self, product_info):
        analysis = {}
        analysis['nutrition'] = self.analyze_nutrition(product_info.nutrition)
        analysis['processing'] = self.analyze_processing(product_info.ingredients)
        analysis['harmful_ingredients'] = self.check_harmful_ingredients(product_info.ingredients)
        analysis['allergens'] = self.check_allergens(product_info.ingredients)
        return analysis

In [5]:

class ProductCatalogue:
    def __init__(self):
        print("Initializing ProductCatalogue...")
        self.db_engine = create_engine('sqlite:///product_catalogue.db', echo=False)
        self.recreate_table()
        self.Session = sessionmaker(bind=self.db_engine)
        self.label_classifier = self.train_label_classifier()
        self.health_analyzer = HealthAnalysis()
        print("ProductCatalogue initialized.")

    def recreate_table(self):
        print("Recreating database tables...")
        Base.metadata.drop_all(self.db_engine)
        Base.metadata.create_all(self.db_engine)
        print("Database tables recreated.")

    def load_cnf_data(self):
        print("Loading CNF data...")
        logger.info("Downloading Canadian Nutrient File data...")
        url = "https://www.canada.ca/content/dam/hc-sc/migration/hc-sc/fn-an/alt_formats/zip/nutrition/fiche-nutri-data/cnf-fcen-csv.zip"
        response = requests.get(url)
        zipfile = ZipFile(BytesIO(response.content))

        logger.info("Processing food name data...")
        food_name_df = pd.read_csv(zipfile.open('FOOD NAME.csv'), encoding='latin1')

        logger.info("Processing nutrient amount data...")
        nutrient_amount_df = pd.read_csv(zipfile.open('NUTRIENT AMOUNT.csv'), encoding='latin1')

        logger.info("Processing nutrient name data...")
        nutrient_name_df = pd.read_csv(zipfile.open('NUTRIENT NAME.csv'), encoding='latin1')

        # Merge nutrient data
        nutrient_df = pd.merge(nutrient_amount_df, nutrient_name_df, on='NutrientID')

        # Pivot the nutrient data to have nutrients as columns
        nutrient_pivot = nutrient_df.pivot(index='FoodID', columns='NutrientName', values='NutrientValue')

        # Merge food name data with nutrient data
        merged_df = pd.merge(food_name_df, nutrient_pivot, left_on='FoodID', right_index=True)

        logger.info("Preparing data for database...")
        products = []
        for _, row in merged_df.iterrows():
            product = {
                'name': row['FoodDescription'],
                'food_group_id': str(row['FoodGroupID']),
                'food_code': str(row['FoodID']),
                'description': row['FoodDescription'],
                'calories': self.safe_float(row.get('ENERGY (KILOCALORIES)')),
                'protein': self.safe_float(row.get('PROTEIN')),
                'carbohydrates': self.safe_float(row.get('CARBOHYDRATE, TOTAL (BY DIFFERENCE)')),
                'fat': self.safe_float(row.get('FAT (TOTAL LIPIDS)')),
                'sugar': self.safe_float(row.get('SUGARS, TOTAL')),
                'sodium': self.safe_float(row.get('SODIUM'))
            }
            products.append(product)

        self.update_database(products)
        print(f"CNF data loaded. {len(products)} products added to the database.")


    @staticmethod
    def safe_float(value):
        try:
            return float(value) if value is not None else None
        except ValueError:
            logger.warning(f"Could not convert {value} to float. Setting to None.")
            return None

    def train_label_classifier(self):
        training_data = [
            ("ENERGY (KILOCALORIES)", "calories"),
            ("PROTEIN", "protein"),
            ("CARBOHYDRATE, TOTAL (BY DIFFERENCE)", "carbohydrates"),
            ("FAT (TOTAL LIPIDS)", "fat"),
            ("SUGARS, TOTAL", "sugar"),
            ("SODIUM", "sodium"),
            ("Food Group", "food_group_id"),
            ("Food Code", "food_code"),
            ("Food Description", "name"),
        ]

        # Duplicate the data to increase the dataset size
        training_data = training_data * 10

        X, y = zip(*training_data)

        # Print class distribution
        logger.info("Class distribution:")
        for label in set(y):
            logger.info(f"{label}: {y.count(label)}")

        vectorizer = TfidfVectorizer()
        X_vectorized = vectorizer.fit_transform(X)

        classifier = MultinomialNB()

        # Use cross-validation to get a more reliable accuracy estimate
        cv_scores = cross_val_score(classifier, X_vectorized, y, cv=5)
        logger.info(f"Cross-validation scores: {cv_scores}")
        logger.info(f"Mean CV accuracy: {np.mean(cv_scores):.2f} (+/- {np.std(cv_scores) * 2:.2f})")

        # Fit the classifier on the entire dataset
        classifier.fit(X_vectorized, y)

        # Print detailed classification report
        y_pred = classifier.predict(X_vectorized)
        logger.info("\nClassification Report:")
        logger.info(classification_report(y, y_pred, zero_division=1))

        return classifier, vectorizer

    def update_database(self, products):

        session = self.Session()
        try:
            for product_data in products:
                # Convert string values to appropriate types
                for key, value in product_data.items():
                    if key in ['calories', 'protein', 'carbohydrates', 'fat', 'sugar', 'sodium']:
                        try:
                            product_data[key] = float(value) if value else None
                        except ValueError:
                            logger.warning(f"Could not convert {key}: {value} to float. Setting to None.")
                            product_data[key] = None

                product = Product(**product_data)
                session.add(product)
            session.commit()
            logger.info(f"Successfully added {len(products)} products to the database.")
            print("Database update complete.")
        except Exception as e:
            session.rollback()
            logger.error(f"An error occurred while updating the database: {str(e)}")
        finally:
            session.close()


    def get_products(self, limit=10):
        print(f"Retrieving up to {limit} products from the database...")
        session = self.Session()
        try:
            products = session.query(Product).limit(limit).all()
            print(f"Retrieved {len(products)} products from the database.")
            return products
        except Exception as e:
            print(f"An error occurred while retrieving products: {str(e)}")
            return []
        finally:
            session.close()

    def display_products(self, limit=10):
        products = self.get_products(limit)
        if products:
            print(f"\nDisplaying {len(products)} products from the database:")
            for product in products:
                print(f"ID: {product.id}")
                print(f"Name: {product.name}")
                print(f"Food Group ID: {product.food_group_id}")
                print(f"Food Code: {product.food_code}")
                print(f"Description: {product.description}")
                print(f"Calories: {product.calories}")
                print(f"Protein: {product.protein}")
                print(f"Carbohydrates: {product.carbohydrates}")
                print(f"Fat: {product.fat}")
                print(f"Sugar: {product.sugar}")
                print(f"Sodium: {product.sodium}")
                print("------------------------")
        else:
            print("No products found in the database.")

    def check_database(self):
        session = self.Session()
        try:
          product_count = session.query(Product).count()
          print(f"Total products in the database: {product_count}")
          if product_count > 0:
            first_product = session.query(Product).first()
            print("First product in the database:")
            print(f"ID: {first_product.id}")
            print(f"Name: {first_product.name}")
        except Exception as e:
          print(f"An error occurred while checking the database: {str(e)}")
        finally:
          session.close()

    def display_products_with_analysis(self, limit=10, user_profile=None):
        products = self.get_products(limit)
        if products:
            print(f"\nDisplaying {len(products)} products from the database with health analysis:")
            for product in products:
                print(f"ID: {product.id}")
                print(f"Name: {product.name}")
                print(f"Food Group ID: {product.food_group_id}")
                print(f"Food Code: {product.food_code}")
                print(f"Description: {product.description}")
                print(f"Calories: {product.calories}")
                print(f"Protein: {product.protein}")
                print(f"Carbohydrates: {product.carbohydrates}")
                print(f"Fat: {product.fat}")
                print(f"Sugar: {product.sugar}")
                print(f"Sodium: {product.sodium}")

                try:
                    analysis = self.health_analyzer.analyze_product(product)
                    print("\nHealth Analysis:")
                    print(f"Nutrition Rating: {analysis['nutrition_rating']}/5")
                    print(f"Processing Level: {analysis['processing_level']}/4")
                    if analysis['harmful_ingredients']:
                        print(f"Harmful Ingredients: {', '.join(analysis['harmful_ingredients'])}")
                    if 'diet_compliant' in analysis:
                        print(f"Diet Compliant: {'Yes' if analysis['diet_compliant'] else 'No'}")
                    if 'allergens' in analysis:
                        if analysis['allergens']:
                            print(f"Allergens: {', '.join(analysis['allergens'])}")
                        else:
                            print("No allergens detected")
                except Exception as e:
                    print(f"Error in health analysis: {str(e)}")
                print("------------------------")
        else:
            print("No products found in the database.")

    def scrape_online_retailers(self):
        print("Scraping online retailers...")
        logger.info("Scraping online retailers...")
        # For this example, we'll use a sample nutrition facts website
        url = "https://www.nutritionvalue.org/"
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            products = []
            food_items = soup.find_all('a', class_='food_link')

            for item in food_items[:5]:  # Limit to 5 items for this example
                product_url = "https://www.nutritionvalue.org" + item['href']
                product_response = requests.get(product_url, headers=headers)
                product_soup = BeautifulSoup(product_response.content, 'html.parser')

                name = item.text.strip()
                calories = product_soup.find('td', text='Calories').find_next('td').text.strip() if product_soup.find('td', text='Calories') else None
                protein = product_soup.find('td', text='Protein').find_next('td').text.strip() if product_soup.find('td', text='Protein') else None
                carbs = product_soup.find('td', text='Carbohydrate').find_next('td').text.strip() if product_soup.find('td', text='Carbohydrate') else None
                fat = product_soup.find('td', text='Total Fat').find_next('td').text.strip() if product_soup.find('td', text='Total Fat') else None
                sugar = product_soup.find('td', text='Sugars, total').find_next('td').text.strip() if product_soup.find('td', text='Sugars, total') else None
                sodium = product_soup.find('td', text='Sodium').find_next('td').text.strip() if product_soup.find('td', text='Sodium') else None

                product = {
                    'name': name,
                    'food_group_id': None,
                    'food_code': None,
                    'description': None,
                    'calories': self.extract_numeric(calories),
                    'protein': self.extract_numeric(protein),
                    'carbohydrates': self.extract_numeric(carbs),
                    'fat': self.extract_numeric(fat),
                    'sugar': self.extract_numeric(sugar),
                    'sodium': self.extract_numeric(sodium)
                }
                products.append(product)
                logger.info(f"Scraped product: {name}")

            logger.info(f"Scraped {len(products)} products from online retailers.")
            print(f"Scraped {len(products)} products from online retailers.")
            return products

        except requests.RequestException as e:
            logger.error(f"An error occurred while scraping: {str(e)}")
            return []

    @staticmethod
    def extract_numeric(value):
        if value:
            return float(''.join(filter(lambda x: x.isdigit() or x == '.', value)))
        return None

    def extract_label_info(self, image_file):
        print("Extracting label information from image...")
        # Read the image file
        file_bytes = np.asarray(bytearray(image_file.read()), dtype=np.uint8)
        image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)

        # Preprocess image
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        denoised = cv2.fastNlMeansDenoising(gray)

        # Perform OCR
        text = pytesseract.image_to_string(denoised)

        # Classify each line of text
        classifier, vectorizer = self.label_classifier
        lines = text.split('\n')
        classified_info = {
            'name': None,
            'food_group_id': None,
            'food_code': None,
            'description': None,
            'calories': None,
            'protein': None,
            'carbohydrates': None,
            'fat': None,
            'sugar': None,
            'sodium': None
        }

        for line in lines:
            if line.strip():
                line_vectorized = vectorizer.transform([line])
                label = classifier.predict(line_vectorized)[0]
                if label in classified_info:
                    classified_info[label] = line.strip()

        print("Label information extracted.")
        return classified_info

    def create_edit_form(self, product_info):
        form_html = """
        <form id="productForm">
        """
        for key, value in product_info.items():
            form_html += f"""
            <label for="{key}">{key.capitalize()}:</label>
            <input type="text" id="{key}" name="{key}" value="{value if value is not None else ''}"><br><br>
            """
        form_html += """
        <button type="button" onclick="submitForm()">Submit</button>
        </form>

        <div id="output"></div>

        <script>
        function submitForm() {
          var form = document.getElementById('productForm');
          var formData = new FormData(form);
          var jsonData = {};
          for (var [key, value] of formData.entries()) {
              jsonData[key] = value;
          }
          document.getElementById('output').innerText = 'Submitting: ' + JSON.stringify(jsonData);
          google.colab.kernel.invokeFunction('notebook.submit_form', [JSON.stringify(jsonData)], {});
        }
        </script>
        """
        display(HTML(form_html))

        def handle_form_submission(jsonData):
            data = json.loads(jsonData)
            self.update_database([data])
            print("Product information submitted to database.")

        output.register_callback('notebook.submit_form', handle_form_submission)

    def run(self):
        print("Starting ProductCatalogue run...")
        try:
            self.load_cnf_data()

            print("Please upload an image of a product label.")
            uploaded = files.upload()

            if not uploaded:
                print("No file was uploaded. Skipping image processing.")
            else:
                for filename, file_data in uploaded.items():
                    print(f"Processing {filename}...")
                    with BytesIO(file_data) as image_file:
                        extracted_info = self.extract_label_info(image_file)
                        print("Extracted information:")
                        for key, value in extracted_info.items():
                            print(f"{key}: {value}")
                        print("Please review and edit the extracted information:")
                        self.create_edit_form(extracted_info)

            print("Scraping online retailers...")
            online_products = self.scrape_online_retailers()
            if online_products:
                print("Adding scraped products to the database...")
                self.update_database(online_products)

            print("Displaying products from the database:")
            self.display_products(limit=10)

            print("Displaying products from the database with health analysis:")
            user_profile = {'diet': 'vegan', 'allergies': ['peanuts', 'soy']}
            self.display_products_with_analysis(limit=10, user_profile=user_profile)

        except Exception as e:
            print(f"An unexpected error occurred: {str(e)}")
            import traceback
            traceback.print_exc()
        print("ProductCatalogue run complete.")

# Example usage
print("Creating ProductCatalogue instance...")
catalogue = ProductCatalogue()
print("Running ProductCatalogue...")
catalogue.run()
print("ProductCatalogue execution complete.")


Creating ProductCatalogue instance...
Initializing ProductCatalogue...
Recreating database tables...
Database tables recreated.
Nutrition Model Performance:
              precision    recall  f1-score   support

           1       0.94      0.99      0.96       175
           2       0.14      0.09      0.11        11
           3       0.43      0.33      0.38         9
           4       1.00      0.20      0.33         5

    accuracy                           0.89       200
   macro avg       0.63      0.40      0.45       200
weighted avg       0.87      0.89      0.87       200

Processing Model Performance:
              precision    recall  f1-score   support

           3       1.00      1.00      1.00        28
           4       1.00      1.00      1.00       172

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200

ProductCatalogue initialized.
Running ProductCatalo

Saving whey-protein.webp to whey-protein.webp
Processing whey-protein.webp...
Extracting label information from image...
Label information extracted.
Extracted information:
name: None
food_group_id: None
food_code: None
description: None
calories: Threonine/Thréonine...
protein: Ingredients: New Zealand whey protein concentrate, Chicory root
carbohydrates: Carbohydrate / Glucides 3 g
fat: Fat / Lipides 2 g 3%
sugar: Sugars / Sucres 2g 2%
sodium: Sodium 140 mg 6%
Please review and edit the extracted information:


Scraping online retailers...
Scraping online retailers...
Scraped 0 products from online retailers.
Displaying products from the database:
Retrieving up to 10 products from the database...
Retrieved 10 products from the database.

Displaying 10 products from the database:
ID: 1
Name: Cheese souffle
Food Group ID: 22
Food Code: 2
Description: Cheese souffle
Calories: 204.0
Protein: 9.54
Carbohydrates: 5.91
Fat: 15.7
Sugar: 2.66
Sodium: 318.0
------------------------
ID: 2
Name: Chop suey, with meat, canned
Food Group ID: 22
Food Code: 4
Description: Chop suey, with meat, canned
Calories: 60.0
Protein: 4.07
Carbohydrates: 5.29
Fat: 2.8
Sugar: 3.4
Sodium: 490.0
------------------------
ID: 3
Name: Chinese dish, chow mein, chicken
Food Group ID: 22
Food Code: 5
Description: Chinese dish, chow mein, chicken
Calories: 85.0
Protein: 6.76
Carbohydrates: 8.29
Fat: 2.8
Sugar: 1.74
Sodium: 311.0
------------------------
ID: 4
Name: Corn fritter
Food Group ID: 18
Food Code: 6
Description: Corn fri

In [6]:
class ProductInfo:
    def __init__(self):
        self.name = ""
        self.ingredients = ""
        self.nutrition = {}
        self.claims = []
        self.extracted_text = ""
        self.additional_info = ""

    def to_dict(self):
        return {
            "name": self.name,
            "ingredients": self.ingredients,
            "nutrition": self.nutrition,
            "claims": self.claims,
            "extracted_text": self.extracted_text,
            "additional_info": self.additional_info
        }

class ComprehensiveChatbot:
    def __init__(self):
        self.product_info = ProductInfo()
        self.health_analyzer = HealthAnalysis()
        self.conversation_history = []
   #    openai.api_key = "API_KEY"

    def extract_info_from_image(self, image):
        try:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
            text = pytesseract.image_to_string(threshold)

            self.product_info.extracted_text = text

            lines = text.split('\n')
            for line in lines:
                line_lower = line.lower()

                if 'product name' in line_lower or 'name:' in line_lower:
                    self.product_info.name = line.split(':', 1)[1].strip() if len(line.split(':', 1)) > 1 else ""
                elif 'ingredients' in line_lower:
                    self.product_info.ingredients = line.split(':', 1)[1].strip() if len(line.split(':', 1)) > 1 else ""
                elif 'calories' in line_lower:
                    self.product_info.nutrition['calories'] = line
                elif 'total fat' in line_lower:
                    self.product_info.nutrition['total_fat'] = line
                elif 'saturated fat' in line_lower:
                    self.product_info.nutrition['saturated_fat'] = line
                elif 'trans fat' in line_lower:
                    self.product_info.nutrition['trans_fat'] = line
                elif 'cholesterol' in line_lower:
                    self.product_info.nutrition['cholesterol'] = line
                elif 'sodium' in line_lower:
                    self.product_info.nutrition['sodium'] = line
                elif 'total carbohydrate' in line_lower:
                    self.product_info.nutrition['total_carbohydrate'] = line
                elif 'dietary fiber' in line_lower:
                    self.product_info.nutrition['dietary_fiber'] = line
                elif 'total sugars' in line_lower:
                    self.product_info.nutrition['total_sugars'] = line
                elif 'protein' in line_lower:
                    self.product_info.nutrition['protein'] = line

                for claim in ['low fat', 'high fiber', 'sugar free', 'organic', 'non-gmo', 'gluten free']:
                    if claim in line_lower:
                        self.product_info.claims.append(line.strip())

            return text
        except Exception as e:
            print(f"Error in extract_info_from_image: {str(e)}")
            return ""

    def get_ai_response(self, user_input):
        try:
            if "health analysis" in user_input.lower():
                health_analysis = self.health_analyzer.analyze_product(self.product_info)
                analysis_str = json.dumps(health_analysis, indent=2)
                response = f"Here's the health analysis for the product:\n{analysis_str}\n\nWould you like me to explain any part of this analysis?"
            else:
                self.conversation_history.append({"role": "user", "content": user_input})

                context = f"""
                Product Information:
                {json.dumps(self.product_info.to_dict(), indent=2)}

                Please answer the user's question based on all the information provided above,
                including the extracted text from the image and any additional information provided by the user.
                If the information is not available or cannot be inferred from the given data, please state that clearly.
                """

                ai_response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[
                        {"role": "system", "content": "You are an AI assistant specialized in analyzing food products and providing health insights based on product labels and additional information."},
                        {"role": "user", "content": context},
                        *self.conversation_history
                    ]
                )

                response = ai_response.choices[0].message['content']
                self.conversation_history.append({"role": "assistant", "content": response})

            return response
        except Exception as e:
            print(f"Error in get_ai_response: {str(e)}")
            return "I'm sorry, an error occurred while processing your request."

    def run(self):
        print("Welcome to the Comprehensive Smart Label Reader Chatbot!")
        print("Please upload an image of a product label.")

        uploaded = files.upload()
        if not uploaded:
            print("No file was uploaded. Please try again.")
            return

        for filename, file_data in uploaded.items():
            print(f"Processing {filename}...")
            file_bytes = np.asarray(bytearray(file_data), dtype=np.uint8)
            image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
            extracted_text = self.extract_info_from_image(image)
            print("Extracted information:")
            print(extracted_text)

        additional_info = input("Please provide any additional ingredients or claim information: ")
        self.product_info.additional_info = additional_info
        self.product_info.claims.extend([claim.strip() for claim in additional_info.split(',')])

        print("\nProduct information extracted. You can now ask questions about the product or request a health analysis.")

        while True:
            human_input = input("\nWhat would you like to know about this product? (Type 'Exit' to quit): ")

            if human_input.lower() == 'exit':
                print("Thank you for using the Comprehensive Smart Label Reader Chatbot. Goodbye!")
                break

            response = self.get_ai_response(human_input)
            print("AI Assistant:", response)

# Example usage
print("Initializing chatbot...")
chatbot = ComprehensiveChatbot()
print("Starting chatbot...")
chatbot.run()





Initializing chatbot...
Nutrition Model Performance:
              precision    recall  f1-score   support

           1       0.94      0.99      0.96       175
           2       0.14      0.09      0.11        11
           3       0.43      0.33      0.38         9
           4       1.00      0.20      0.33         5

    accuracy                           0.89       200
   macro avg       0.63      0.40      0.45       200
weighted avg       0.87      0.89      0.87       200

Processing Model Performance:
              precision    recall  f1-score   support

           3       1.00      1.00      1.00        28
           4       1.00      1.00      1.00       172

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200

Starting chatbot...
Welcome to the Comprehensive Smart Label Reader Chatbot!
Please upload an image of a product label.


Saving whey-protein.webp to whey-protein (1).webp
Processing whey-protein (1).webp...
Extracted information:
Precision All Natural™ is an innovative line
of gluten free protein drink mixes formulated
without any artificial flavours, colours

and sweeteners.

 

 

Nutrition Facts

Valeur nutritive

Per 1 scoop (35 g)
Par 1 mesure (35 g)

Calories 140

‘% Dally Value*
% valeur quotidienne*

Fat / Lipides 2 g 3%
Saturated / saturés 1.5 g 8%
+ Trans /trans 0,1 g

Carbohydrate / Glucides 3 g
Fibre / Fibres 1g 4%
Sugars / Sucres 2 g 2%

Protein / Protéines 27 g

Cholesterol / Cholestérol 65 mg

Sodium 140 mg 6%

Potassium 175 mg 4%

Calcium 100 mg 8%

Iron / Fer 0.2 mg 1%

LT

*5% or less is a little, 15% or more is a lot
*5% ou moins c'est peu, 18% ou plus c'est beaucoup

 

 

Precision All Natural"* est une gamme
innovante de mélanges de boissons protéinées
sans gluten et exempts de saveurs, couleurs ou
édulcorants artificiels.

Suggested Use: Precision All Natural™ Whey Protein can be
e